In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
plt.rc('font', family='Verdana')

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV

import catboost

In [2]:
pd.set_option('display.max_columns', 60)

data = DataFrame.from_csv('data/processed_data_version_1.csv', encoding='utf-8')
data.head()

,TARGET,AGE,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,OWN_AUTO,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,WORK_TIME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,SOLVENCY,SOCSTATUS_WORK_FL_0,SOCSTATUS_WORK_FL_1,SOCSTATUS_PENS_FL_0,SOCSTATUS_PENS_FL_1,GENDER_0,GENDER_1,EDUCATION_Высшее,EDUCATION_Неоконченное высшее,EDUCATION_Неполное среднее,EDUCATION_Среднее,...,REG_FACT_FL_0,REG_FACT_FL_1,FACT_POST_FL_0,FACT_POST_FL_1,REG_POST_FL_0,REG_POST_FL_1,REG_FACT_POST_FL_0,REG_FACT_POST_FL_1,REG_FACT_POST_TP_FL_0,REG_FACT_POST_TP_FL_1,FL_PRESENCE_FL_0,FL_PRESENCE_FL_1,AUTO_RUS_FL_0,AUTO_RUS_FL_1,HS_PRESENCE_FL_0,HS_PRESENCE_FL_1,COT_PRESENCE_FL_0,COT_PRESENCE_FL_1,GAR_PRESENCE_FL_0,GAR_PRESENCE_FL_1,LAND_PRESENCE_FL_0,LAND_PRESENCE_FL_1,GPF_DOCUMENT_FL_0,GPF_DOCUMENT_FL_1,FACT_PHONE_FL_0,FACT_PHONE_FL_1,REG_PHONE_FL_0,REG_PHONE_FL_1,GEN_PHONE_FL_0,GEN_PHONE_FL_1
0,0,49,2,1,5000.0,0,8000.00,6,8650.0,220,18.0,1,1,6,2,1,1580.000000,1580.0,0.0,3.750000,0,1,1,0,0,1,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1
1,0,32,3,3,12000.0,0,21650.00,6,4000.0,137,97.0,1,1,6,1,1,4020.000000,4020.0,0.0,3.325635,0,1,1,0,0,1,0,0,0,1,...,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1
2,0,52,4,0,9000.0,0,33126.00,12,4000.0,251,84.0,2,1,11,0,0,0.000000,0.0,0.0,3.260279,0,1,1,0,0,1,0,0,1,0,...,0,1,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0,1
3,0,39,1,1,25000.0,0,8491.82,6,5000.0,36,168.0,1,1,6,3,1,1589.923333,1590.0,0.0,17.664058,0,1,1,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1
4,0,30,0,0,12000.0,0,21990.00,12,4000.0,83,101.0,2,1,16,2,1,1152.150000,2230.0,0.0,6.548431,0,1,1,0,1,0,0,0,0,1,...,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0,0,1


In [3]:
for i in ['TARGET', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'GENDER', 
      'EDUCATION', 'MARITAL_STATUS', 'GEN_INDUSTRY', 'GEN_TITLE',
      'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR', 'REG_ADDRESS_PROVINCE',
      'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE', 'REGION_NM',
      'REG_FACT_FL', 'FACT_POST_FL', 'REG_POST_FL', 'REG_FACT_POST_FL', 
      'REG_FACT_POST_TP_FL', 'FL_PRESENCE_FL', 'AUTO_RUS_FL', 'HS_PRESENCE_FL',
      'COT_PRESENCE_FL', 'GAR_PRESENCE_FL', 'LAND_PRESENCE_FL', 'DL_DOCUMENT_FL',
      'GPF_DOCUMENT_FL', 'FACT_PHONE_FL', 'REG_PHONE_FL', 'GEN_PHONE_FL']:
    if i in data.columns:
        data[i]=data[i].astype('str')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15223 entries, 0 to 15222
Columns: 442 entries, TARGET to GEN_PHONE_FL_1
dtypes: float64(8), int64(434)
memory usage: 51.5 MB


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('TARGET', axis=1), data['TARGET'], test_size=0.3,
                                                    random_state=42)

In [5]:
cat = catboost.CatBoostClassifier(random_seed=42)

In [6]:
cat.fit(X_train, y_train)

In [7]:
cat.score(X=X_test, y=y_test)

0.87825706152835559

In [8]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [10]:
roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])

0.57321683597855089

In [12]:
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 25, 50, 100, 150, 200, 250, 300]

for ntrees in trees_grid:
    rf = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in strat.split(X_train, y_train):
        Xtrain, Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
        ytrain, ytest = y_test[train_index], y_test[test_index]
        rf.fit(Xtrain, ytrain)
        temp_train_acc.append(rf.score(Xtrain, ytrain))
        temp_test_acc.append(rf.score(Xtest, ytest))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Наилучшее значение правильности перекрестной проверки равно {:.2f}% и достигается при {} деревьях".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
plt.style.use('ggplot')

fig, ax = plt.subplots(figsize=(16, 10))
ax.plot(trees_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', 
        label='обучение')
ax.plot(trees_grid, test_acc.mean(axis=1), alpha=0.5, color='red', 
        label='перекрестная проверка')
ax.fill_between(trees_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.85,1.01])
ax.set_ylabel("Правильность")
ax.set_xlabel("Количество деревьев")

In [16]:
param_grid = {'max_features': [3, 4, 5, 6, 7, 8, 9, 12, 15, 18, 21], 'max_depth': [6, 8, 10, 12, 14, 16, 18, 20, 22]}

In [ ]:
rf = RandomForestClassifier(n_estimators=500, random_state=42)
grid_search = GridSearchCV(rf, param_grid, n_jobs=-1, cv=strat)
grid_search.fit(X_train, y_train)

In [ ]:
test_score = roc_auc_score(y_test, grid_search.predict(X_test))
print("AUC на тестовой выборке: {:.2f}".format(test_score))
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))

In [19]:
forest2 = RandomForestClassifier(n_estimators=500, max_depth=12, random_state=152, n_jobs=-1)
forest2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=-1, oob_score=False, random_state=152,
            verbose=0, warm_start=False)

In [20]:
roc_auc_score(y_test, forest2.predict_proba(X_test)[:, 1])

0.69661016949152543